In [1]:
import requests
from bs4 import BeautifulSoup as bs
import yaml
from IPython.display import display
from time import sleep
import re



In [2]:
from typing import List, Dict, Any, Set, Tuple, Union, Optional

In [3]:
url = "http://factcheck.snu.ac.kr/v2/facts/%d"

In [4]:
responce = requests.get(url % 3640)

In [5]:
responce.status_code

200

In [6]:
soup = bs(responce.text, 'html.parser')

In [4]:

# SNU 팩트체크 사이트는 다음과 같이 팩트 체크 별 아이디와 점수를 보냅니다.
# <ul> <li class="fcItem_vf_li">...</li>
# <script charset="utf-8" type="text/javascript">
# $(function () { showScore(아이디, 점수)}); </script> ... </ul>
# 따라서 아이디와 점수를 추출하는 정규표현식을 사용합니다.

id_score_re = re.compile(r'(?<=showScore\()\d+, \d+')
  
# 아이디와 점수를 추출하는 함수를 정의합니다.

def get_fc_id_score(soup:bs) -> List[Tuple[int, int]]:
    id_score: List[Tuple[int,int]] = [] # 아이디와 점수를 저장할 리스트
    for fc_item_script in soup.select(".fcItem_vf > ul > script"): # 팩트 체크 아이템의 스크립트를 찾습니다.
        id_score_str: str = id_score_re.search(fc_item_script.text).group() # 정규 표현식으로 아이디와 점수값을 담은 문자열을 추출합니다.
        id_score_int: Tuple = tuple(map(int, id_score_str.split(', '))) # 문자열을 정수로 변환합니다.
        id_score.append(id_score_int)
    return id_score


whitespace_re = re.compile(r'\s{2,}') # 공백 제거용 정규표현식을 정의합니다.

# 작성 시간과 내용을 추출하는 함수를 정의합니다.

def get_fc_time_contents(soup:bs) -> List[Tuple[str, str, str]]:
    time_contents: List[str] = [] # 시간과 내용을 저장할 리스트
    for fc_item in soup.select(".fcItem_vf > ul > li"): # 팩트 체크 아이템을 찾습니다.
        date = fc_item.select_one(".reg_date > p i:nth-child(1)").text # 작성 날짜를 추출합니다.
        time = fc_item.select_one(".reg_date > p i:nth-child(2)").text # 작성 시간을 추출합니다.
        raw_content = fc_item.select_one(".vf_exp_wrap").text # 팩트 체크 내용을 추출합니다.
        content = re.sub(whitespace_re, " ", raw_content.strip()) # 공백을 제거합니다.
        time_contents.append((date, time, content))
    return time_contents

# 위의 함수들의 반환을 합쳐 최종적으로 아이디를 키로 갖고 나머지 내용을 값으로 갖는 딕셔너리를 반환하는 함수를 정의합니다.

def get_fc(soup:bs) -> Dict[int, Dict[Any, Any]]:
    id_score = get_fc_id_score(soup) # 아이디와 점수를 추출합니다.
    time_contents = get_fc_time_contents(soup) # 작성 시간과 내용을 추출합니다.
    zipped = zip(id_score, time_contents) # 두 리스트를 합쳐 쌍을 생성합니다.
    return {id: {'score': score, 'date': date, 'time': time, 'content': content} for (id, score), (date, time, content) in zipped}

In [7]:
res149 = requests.get(url % 149)
soup149 = bs(res149.text, 'html.parser')

In [10]:
get_fc(soup149)

{188: {'score': 1,
  'date': '2017.04.28',
  'time': '22:34',
  'content': '검증내용 사실이 아니다. 지니계수는 소득분배 불평등을 나타내는 수치로 0에 가까울수록 소득 분배가 평등하고 1에 가까울수록 불평등함을 나타낸다.\xa0통계청 가계동향조사를 보면 노무현 정부 시기(2003년 2월~2008년 2월) 동안의 지니계수는 도시 2인이상 가구 시장소득 기준으로 갈수록 상승했다. 그러나 이명박 대통령이 취임한 해인 2008년 더 상승, 2009년 정점을 찍었다. 2006년부터 조사된 전체가구 시장소득 기준 지니계수도 이명박 정부 취임 이후에 더 올랐다. 시장소득이 아닌 처분가능 소득을 기준으로 해도 노무현 정부보다는 이명박 정부에서 지니계수가 더 높게 나타난다. 검증기사 [팩트파인더] 노무현 정부 때 지니계수 최악이었다?'},
 194: {'score': 2,
  'date': '2017.04.29',
  'time': '16:21',
  'content': '검증내용 2인 이상 가구를 기준으로 했을 때 우리나라의 지니계수가 가장 높았던 시기는 2009년으로 노무현 정부(2003년 2월~ 2008년 2월)가 아닌,\xa0이명박 전 대통령 재임기간이다. 따라서 노무현 정부 때 지니계수가 가장 나빴다는 홍준표 후보의 발언은 틀린 말이다.\xa0하지만 노무현 정부 시절 지니계수가 꾸준히 악화된 게 사실이고, 전체가구 기준으로는 노 전 대통령과 이 전 대통령의 임기가 겹쳤던\xa02008년과 이 전 대통령 시절인 2009년의 수치가 같다. 따라서 노 전대통령의 경제정책 실패로 지수가 계속 상승해 이명박 정부 초반에 정점을 찍었다는 주장이라면 어느정도 홍 후보의 주장이 설득력이 있어 대체로 거짓이라고 할 수 있다. 검증기사 [팩트체크] 지니계수, 노무현 대통령 때 가장 높았다? 근거자료 1 : 링크(제목없음)'},
 209: {'score': 1,
  'date': '2017.05.01',
  'time': '15

In [13]:

yaml.dump(get_fc(soup149), allow_unicode=True)

'188:\n  content: 검증내용 사실이 아니다. 지니계수는 소득분배 불평등을 나타내는 수치로 0에 가까울수록 소득 분배가 평등하고 1에 가까울수록 불평등함을\n    나타낸다.\xa0통계청 가계동향조사를 보면 노무현 정부 시기(2003년 2월~2008년 2월) 동안의 지니계수는 도시 2인이상 가구 시장소득 기준으로\n    갈수록 상승했다. 그러나 이명박 대통령이 취임한 해인 2008년 더 상승, 2009년 정점을 찍었다. 2006년부터 조사된 전체가구 시장소득\n    기준 지니계수도 이명박 정부 취임 이후에 더 올랐다. 시장소득이 아닌 처분가능 소득을 기준으로 해도 노무현 정부보다는 이명박 정부에서 지니계수가\n    더 높게 나타난다. 검증기사 [팩트파인더] 노무현 정부 때 지니계수 최악이었다?\n  date: 2017.04.28\n  score: 1\n  time: \'22:34\'\n194:\n  content: \'검증내용 2인 이상 가구를 기준으로 했을 때 우리나라의 지니계수가 가장 높았던 시기는 2009년으로 노무현 정부(2003년 2월~\n    2008년 2월)가 아닌,\xa0이명박 전 대통령 재임기간이다. 따라서 노무현 정부 때 지니계수가 가장 나빴다는 홍준표 후보의 발언은 틀린 말이다.\xa0하지만\n    노무현 정부 시절 지니계수가 꾸준히 악화된 게 사실이고, 전체가구 기준으로는 노 전 대통령과 이 전 대통령의 임기가 겹쳤던\xa02008년과 이\n    전 대통령 시절인 2009년의 수치가 같다. 따라서 노 전대통령의 경제정책 실패로 지수가 계속 상승해 이명박 정부 초반에 정점을 찍었다는 주장이라면\n    어느정도 홍 후보의 주장이 설득력이 있어 대체로 거짓이라고 할 수 있다. 검증기사 [팩트체크] 지니계수, 노무현 대통령 때 가장 높았다? 근거자료\n    1 : 링크(제목없음)\'\n  date: 2017.04.29\n  score: 2\n  time: \'16:21\'\n209:\n  content: \'검증내용 지니

In [10]:
news_detail = soup149.select_one(".fcItem_detail_top")
news_speacker = news_detail.select_one(".name").text
news_title = news_detail.select_one(".fcItem_detail_li_p > p:nth-child(1) > a").text
news_source_element = news_detail.select_one(".source")
news_source = {news_source_element.text: ""} if news_source_element.select_one("a") == None else {news_source_element.select_one("a").text: news_source_element.select_one("a")['href']}
news_cartegory = [li.text for li in news_detail.select(".fcItem_detail_bottom li")]

news_explain = soup149.select_one(".exp").text
news_speacker, news_title, news_source, news_cartegory, news_explain

('홍준표',
 '홍준표 "소득분배 지니계수가 노무현 정부 때 가장 나빴다"',
 {'\n                      출처 : 28일 중앙선방송토론위원회 주관 대선 후보 TV토론회\n                  ': ''},
 ['정치인(공직자)의 발언', '정치, 19대 대선, 대통령 선거'],
 '\n\n\n\n')

In [9]:
news_source = news_detail.select_one(".source")
news_source = {news_source.text: ""} if news_source.select_one("a") == None else {news_source.select_one("a").text: news_source.select_one("a")['href']}


In [11]:
res3632 = requests.get(url % 3632)
soup3632 = bs(res3632.text, 'html.parser')
news_detail = soup3632.select_one(".fcItem_detail_top")
news_source = news_detail.select_one(".source")
news_source.text if news_source.select_one("a") == None else {news_source.select_one("a").text: news_source.select_one("a")['href']}


{'2021.12.14. 블로그': 'https://blog.naver.com/713sim/222595265146'}

In [12]:
news_detail = soup3632.select_one(".fcItem_detail_top")
news_speacker = news_detail.select_one(".name").text
news_title = news_detail.select_one(".fcItem_detail_li_p > p:nth-child(1) > a").text
news_source_element = news_detail.select_one(".source")
news_source = news_source_element.text if news_source_element.select_one("a") == None else {news_source_element.select_one("a").text: news_source_element.select_one("a")['href']}
news_cartegory = [li.text for li in news_detail.select(".fcItem_detail_bottom li")]

news_explain = soup3632.select_one(".exp").text
news_speacker, news_title, news_source, news_cartegory, news_explain

('심상정',
 '문 정부에서 핵발전소 용량이 증가했고, 4기의 핵발전소가 건설되고 있다',
 {'2021.12.14. 블로그': 'https://blog.naver.com/713sim/222595265146'},
 ['정치인(공직자)의 발언', '정치, 사회, 20대 대통령 선거'],
 '\n            심상정 정의당 대선후보는 지난 14일 블로그에서 “오히려 문재인 정부 임기 동안 핵발전소 용량은 더 증가했고, 지금도 4기의 핵발전소를 건설하고 있다”고 주장했다. 이번 정부가 ‘탈원전’을 표방했지만 제대로 시행하지 않았다는 것이다.\xa0 \n            \n\n\n')

In [23]:

class Speaking:
    def __init__(self, num:int):
        self.num = num
        self.soup = bs(requests.get(url % self.num).text, 'html.parser') # bs를 이용하여 구문 분석 후 soup에 저장합니다.
        self.detail = self.soup.select_one(".fcItem_detail_top") # 상세 페이지를 추출합니다.
        self.speacker = self.detail.select_one(".name").text # 발언자를 추출합니다.
        self.title = self.detail.select_one(".fcItem_detail_li_p > p:nth-child(1) > a").text # 제목을 추출합니다.
        self.source_element = self.detail.select_one(".source") # 출처가 담긴 html 요소를 추출합니다.
        self.source = {self.source_element.text.strip(): ""} \
            if self.source_element.select_one("a") == None\
            else {self.source_element.select_one("a").text.strip(): self.source_element.select_one("a")['href']} # 출처를 추출합니다.
        self.cartegories = {li.text for li in self.detail.select(".fcItem_detail_bottom li")} # 카테고리를 추출합니다.
        self.explain = self.soup.select_one(".exp").text.strip() # 설명을 추출합니다.
        self.factchecks = get_fc(self.soup)
    
    def as_dict(self): # 데이터를 딕셔너리 형태로 반환합니다.
        return {
            'speacker': self.speacker,
            'title': self.title,
            'source': self.source,
            'cartegories': self.cartegories,
            'explain': self.explain,
            'factchecks': self.factchecks
        }
    
    def as_yaml(self): # 데이터를 yaml 형태로 반환합니다.
        return yaml.dump(self.as_dict(), allow_unicode=True)

    def save_as_yaml(self, path:str):
        with open(path, 'w') as f:
            f.write(self.as_yaml())

    # SNU 팩트체크 사이트는 다음과 같이 팩트 체크 별 아이디와 점수를 보냅니다.
    # <ul> <li class="fcItem_vf_li">...</li>
    # <script charset="utf-8" type="text/javascript">
    # $(function () { showScore(아이디, 점수)}); </script> ... </ul>
    # 따라서 아이디와 점수를 추출하는 정규표현식을 사용합니다.

    
    # 아이디와 점수를 추출하는 함수를 정의합니다.

    def get_fc_id_score(soup:bs) -> List[Tuple[int, int]]:
        id_score: List[Tuple[int,int]] = [] # 아이디와 점수를 저장할 리스트
        id_score_re = re.compile(r'(?<=showScore\()\d+, \d+')
        for fc_item_script in soup.select(".fcItem_vf > ul > script"): # 팩트 체크 아이템의 스크립트를 찾습니다.
            id_score_str: str = id_score_re.search(fc_item_script.text).group() # 정규 표현식으로 아이디와 점수값을 담은 문자열을 추출합니다.
            id_score_int: Tuple = tuple(map(int, id_score_str.split(', '))) # 문자열을 정수로 변환합니다.
            id_score.append(id_score_int)
        return id_score



    # 작성 시간과 내용을 추출하는 함수를 정의합니다.

    def get_fc_time_contents(soup:bs) -> List[Tuple[str, str, str]]:
        time_contents: List[str] = [] # 시간과 내용을 저장할 리스트
        for fc_item in soup.select(".fcItem_vf > ul > li"): # 팩트 체크 아이템을 찾습니다.
            date = fc_item.select_one(".reg_date > p i:nth-child(1)").text # 작성 날짜를 추출합니다.
            time = fc_item.select_one(".reg_date > p i:nth-child(2)").text # 작성 시간을 추출합니다.
            raw_content = fc_item.select_one(".vf_exp_wrap").text # 팩트 체크 내용을 추출합니다.
            content = re.sub(r'\s{2,}', " ", raw_content.strip()) # 공백을 제거합니다.
            time_contents.append((date, time, content))
        return time_contents

    # 위의 함수들의 반환을 합쳐 최종적으로 아이디를 키로 갖고 나머지 내용을 값으로 갖는 딕셔너리를 반환하는 함수를 정의합니다.

    def get_fc(soup:bs) -> Dict[int, Dict[Any, Any]]:
        id_score = get_fc_id_score(soup) # 아이디와 점수를 추출합니다.
        time_contents = get_fc_time_contents(soup) # 작성 시간과 내용을 추출합니다.
        zipped = zip(id_score, time_contents) # 두 리스트를 합쳐 쌍을 생성합니다.
        return {id: {'score': score, 'date': date, 'time': time, 'content': content} for (id, score), (date, time, content) in zipped}

news3632 = Speaking(3632)

In [24]:
news3632.as_yaml()

"cartegories: !!set\n  정치, 사회, 20대 대통령 선거: null\n  정치인(공직자)의 발언: null\nexplain: 심상정 정의당 대선후보는 지난 14일 블로그에서 “오히려 문재인 정부 임기 동안 핵발전소 용량은 더 증가했고, 지금도 4기의 핵발전소를\n  건설하고 있다”고 주장했다. 이번 정부가 ‘탈원전’을 표방했지만 제대로 시행하지 않았다는 것이다.\nfactchecks:\n  12579:\n    content: '검증내용 [검증 대상]심상정 정의당 후보의 “오히려 문재인 정부 임기 동안 핵발전소 용량은 더 증가했고, 지금도 4기의 핵발전소를\n      건설하고 있다” 발언[검증 방법]사례 확인, 국내 관련 보고서 검토[검증 내용]심상정 정의당 대선후보는 지난 14일 블로그에서 “오히려\n      문재인 정부 임기 동안 핵발전소 용량은 더 증가했고, 지금도 4기의 핵발전소를 건설하고 있다”고 주장했다. 이번 정부가 ‘탈원전’을 표방했지만\n      제대로 시행하지 않았다는 것이다.\xa0우선, 이번 정부 동안 원전의 설비총량이 증가한 것은 사실이다. 한국전력공사에 따르면, 문재인 정부 초기인\n      2017년의 원전 설비총량은 2017년 2만2천MW에서 2020년 2만3천MW로 증가했다. 2018년 2만1천MW로 잠시 하락했으나 2019년부터\n      2만3천대를 유지했다.(출처=2020년 한국전력통계)설비용량뿐 아니라 원전 발전량과 이용률 또한 상승했다. 원전 발전량은 2017년 14만8천GWh에서\n      2020년 16만GWh로 증가했다. 전체 발전량 중 원전 비중이 2017년 25%에서 2020년 27%로 상승한 것이다. 한국수력원자력에\n      따르면, 원전 이용률 또한 2017년 71.2%에서 2020년 75.3%로 증가했다.(출처=2020년 한국전력통계)이러한 수치 증가는 이전에\n      건설 중이던 원전이 새로 가동됐기 때문이다. 2017년 신고리 3호기가 본격적인 운영에 들어갔고, 201